# Results - Classification of simulated events
This notebook is the primary source of plots and tables for the thesis, with the goal of
keeping every table and figure as standardized as possible. (And who has the time to update
9 million tables one by one anyway).

## TODO
* Descriptive statistics
    - Should descriptive statistics of the simulated data be included?\
    If so, how much? And should it be included for each fold in the k-fold cross-validation?


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from master_scripts.data_functions import get_git_root
from master_scripts.analysis_functions import load_experiment
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
# Load logistic regression experiment
log_ex_id = "506935fb625e"
log_ex = load_experiment(log_ex_id)
#log_model = tf.keras.models.load_model(repo_root + "models/" + c_experiment_id + ".h5")
# Print experiment metrics
print("==== Experiment metrics")
print(json.dumps(log_ex['metrics'], indent=2))
print("====")

==== Experiment metrics
{
  "accuracy_score": 0.7416315789473684,
  "confusion_matrix": {
    "TN": 194025,
    "FP": 43507,
    "FN": 79218,
    "TP": 158250
  },
  "f1_score": 0.7205873982582959,
  "matthews_corrcoef": 0.48881019350314264,
  "roc_auc_score": 0.833443392522035
}
====
